# Iterative methods for solving GEE

In [33]:
import numpy as np
import pandas as pd
from typing import NamedTuple

class CovarianceParameters(NamedTuple('CovarianceParameters', [
    ('alpha', np.float64),
    ('sigma2', np.float64),
])):
    def make_correlation_matrix(self, size):
        correlation_matrix = np.eye(size)
        for i in range(size - 1):
            for j in range(i + 1, size):
                correlation_matrix[i, j] = correlation_matrix[j, i] = np.power(self.alpha, j - i)
        return correlation_matrix
    
    def make_covariance_matrix(self, size):
        return self.make_correlation_matrix(size)*self.sigma2
    
def estimate_covariance_parameters(X, y, beta):
    epsilon = y - X.dot(beta)        
    covariance_matrix = np.outer(epsilon, epsilon)
    
    sigma2 = np.mean(np.diag(covariance_matrix))
    correlation_matrix = covariance_matrix/sigma2
    rho = np.mean([correlation_matrix[i, i + 1] for i in range(correlation_matrix)])
    return CovarianceParameters(alpha=rho, sigma2=sigma2)

In [13]:
orthodont_data = pd.read_csv('orthodont.csv')
orthodont_data = orthodont_data.set_index('Subject')
orthodont_data.head(8)

,distance,age,Sex
Subject,,,
M01,26.0,8,Male
M01,25.0,10,Male
M01,29.0,12,Male
M01,31.0,14,Male
M02,21.5,8,Male
M02,22.5,10,Male
M02,23.0,12,Male
M02,26.5,14,Male


In [24]:
def make_covariates(data_frame):    
    age = (data_frame['age'] - 8).values
    is_female = (data_frame['Sex'] == 'Female').values.astype(np.float64)
    return np.column_stack((
        np.ones(len(data_frame)),
        is_female,
        age,
        age*is_female,        
    ))

def make_response(data_frame):
    return data_frame['distance'].values